In [3]:
import os
os.chdir(r'C:\Users\Josh\Documents\Git\crewai')

from dotenv import load_dotenv
load_dotenv()

from autogen import AssistantAgent, UserProxyAgent, GroupChatManager, GroupChat

config_list = [

    # {
    #     "model": "gpt-4",
    #     "api_key": os.environ.get("AZURE_OPENAI_API_KEY"),
    #     "api_type": "azure",
    #     "base_url": os.environ.get("AZURE_OPENAI_API_BASE"),
    #     "api_version": "2023-12-01-preview",
    # },

    {
        "model": "gpt-4",
        "api_key": os.environ.get("OPENAI_API_KEY"),
        "api_type": "openai",
        "temperature": 0
        
    }
]



![agents.png](agents.png)

In [4]:

# assistant = AssistantAgent("assistant", llm_config={"config_list": config_list})

# user_proxy = UserProxyAgent(
#     "user_proxy", code_execution_config={"work_dir": "coding", "use_docker": False} # IMPORTANT: set to True to run code in docker, recommended

# )  

# user_proxy.initiate_chat(assistant, message="Plot a chart of NVDA and TESLA stock price change YTD.")

In [5]:
llm_config = {"config_list": config_list, "cache_seed": 1988}

user_proxy = UserProxyAgent(
    name="User_proxy",
    system_message="""You are the client admin who liases with the producer owner to ensure alignment\
  """,
    code_execution_config={
        "last_n_messages": 5,
        "work_dir": "groupchat",
        "use_docker": False,
    },  # Please set use_docker=True if docker is available to run the generated code. Using docker is safer than running the generated code directly.
    human_input_mode="ALWAYS",
)

po = AssistantAgent(
    name="Product_owner",
    system_message="""You are a Product Owner in a dynamic software development environment.
  Your primary responsibility is orchestrating the vision of the product, defining
  user stories, and prioritizing the product backlog. You act as the liaison between
  the stakeholders and the development team, ensuring the client's voice is heard
  and reflected in the product development.""",
    llm_config=llm_config,
)

sd1 = AssistantAgent(
    name="Snr_Software_developer",
    system_message="""You are a Senior Developer with extensive experience in software \
  development. You have mastered multiple programming languages and frameworks.\
  Your role involves not only writing clean and efficient code but also architecting \
  scalable software solutions. You are also responsible for guiding and mentoring \
  junior developers, ensuring best practices are followed throughout the development \
  process.""",
    llm_config=llm_config,
)

sd2 = AssistantAgent(
    name="Jnr_Software_developer",
    system_message="""As a Junior Developer, you are at the beginning of your software \
  development career. You are eager to learn and grow your skills in coding, debugging, \
  and collaborating on software projects. Under the mentorship of more experienced \
  developers, you work on various tasks, aiming to contribute effectively to the \
  team's objectives while enhancing your development proficiency.""",
    llm_config=llm_config,
)

sm = AssistantAgent(
    name="Scrum_master",
    system_message="""As a Scrum Master, you are the team's coach and facilitator in an Agile \
  development environment. Your role is to support the Scrum Framework, assisting \
  both the development team and the product owner. You help remove impediments, ensure \
  that Scrum practices are followed, and strive to improve the team's efficiency and \
  effectiveness.""",
    llm_config=llm_config,
)
groupchat = GroupChat(agents=[user_proxy, po, sd1, sd2, sm], messages=[], max_round=12)
manager = GroupChatManager(groupchat=groupchat, llm_config=llm_config)

In [6]:
task1 = """

  Create a detailed project roadmap outlining major milestones, sprint plans with specific goals, \
  user stories, and acceptance criteria for each feature. To do this, you must first clarify the scope of the project with the client, \
  given their initial brief (an app that should allow the client to upload job specifications and \
  a maximum of 100 resumes, then have a large language model like gpt4 score each CV and present the results on an interactive dashboard for candidate comparison.) \
. Collaborate closely with the development team and stakeholders \
  to refine requirements and ensure alignment with client expectations. Document all planning and execution strategies in a clear and accessible format, \
  ensuring transparency and stakeholder engagement. Take an agile approach. Think step by step. """

In [7]:
t1 = user_proxy.initiate_chat(
    manager, message=task1,
    summary_method="reflection_with_llm"
)

User_proxy (to chat_manager):



  Create a detailed project roadmap outlining major milestones, sprint plans with specific goals,   user stories, and acceptance criteria for each feature. To do this, you must first clarify the scope of the project with the client,   given their initial brief (an app that should allow the client to upload job specifications and   a maximum of 100 resumes, then have a large language model like gpt4 score each CV and present the results on an interactive dashboard for candidate comparison.) . Collaborate closely with the development team and stakeholders   to refine requirements and ensure alignment with client expectations. Document all planning and execution strategies in a clear and accessible format,   ensuring transparency and stakeholder engagement. Take an agile approach. Think step by step. 

--------------------------------------------------------------------------------
Product_owner (to chat_manager):

As a Product Owner, I will start by condu

In [8]:
dir(t1)

['__annotations__',
 '__class__',
 '__dataclass_fields__',
 '__dataclass_params__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__match_args__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'chat_history',
 'cost',
 'human_input',
 'summary']

In [11]:
t1.chat_history

[{'content': '\n\n  Create a detailed project roadmap outlining major milestones, sprint plans with specific goals,   user stories, and acceptance criteria for each feature. To do this, you must first clarify the scope of the project with the client,   given their initial brief (an app that should allow the client to upload job specifications and   a maximum of 100 resumes, then have a large language model like gpt4 score each CV and present the results on an interactive dashboard for candidate comparison.) . Collaborate closely with the development team and stakeholders   to refine requirements and ensure alignment with client expectations. Document all planning and execution strategies in a clear and accessible format,   ensuring transparency and stakeholder engagement. Take an agile approach. Think step by step. ',
  'role': 'assistant'},
 {'content': "As a Product Owner, I will start by conducting a series of meetings with the client to understand their needs and expectations in de

In [12]:
task2 = f"""Implement the project plan decided on ({t1.chat_history}) through a simulated week-by-week sprint execution, ALWAYS making sure to write and show your code for EVERY sprint using the tools provided. \
  Use Python, with Flask for the backend and React for the frontend. Integrate OpenAI and 'langchain' or 'llamaindex' \
  for large language model capabilities. Throughout the each sprint, create intermediate artefacts such \
  as code repositories, documentation, and progress reports to maintain transparency with the senior developer and scrum \
  master. Address challenges proactively and seek guidance as needed to ensure adherence to the timeline and quality standards. \
  Your deliverables include the complete application code for EVERY sprint, a detailed development writeup explaining the architecture, \
  technologies used, and a reflection on the development process, including lessons learned and challenges overcome."""

user_proxy.initiate_chat(
    manager, message=task2,
    summary_method="reflection_with_llm"
)

User_proxy (to chat_manager):

Implement the project plan decided on ([{'content': '\n\n  Create a detailed project roadmap outlining major milestones, sprint plans with specific goals,   user stories, and acceptance criteria for each feature. To do this, you must first clarify the scope of the project with the client,   given their initial brief (an app that should allow the client to upload job specifications and   a maximum of 100 resumes, then have a large language model like gpt4 score each CV and present the results on an interactive dashboard for candidate comparison.) . Collaborate closely with the development team and stakeholders   to refine requirements and ensure alignment with client expectations. Document all planning and execution strategies in a clear and accessible format,   ensuring transparency and stakeholder engagement. Take an agile approach. Think step by step. ', 'role': 'assistant'}, {'content': "As a Product Owner, I will start by conducting a series of meetin